In [1]:
!pip install transformers

In [4]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from transformers import BertTokenizerFast, AlbertModel, BertModel, AutoTokenizer
from torch import nn

In [5]:
from google.colab import drive
drive.mount('/content/drive')
os.listdir("/content/drive/My Drive/3조")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['1.기획안&WBS',
 '2.수행일지',
 '3.최종포트폴리오',
 '4.과제',
 '멘토링',
 '회의록',
 '[D11&12]기업요구사항 기반의 문제해결 프로젝트.pdf',
 'Data',
 '빅데이터분석프로젝트_단계별공정.pdf',
 'DS_folder',
 'DE']

In [6]:
PATH = '/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/'

In [36]:
test = pd.read_csv(PATH + 'novel/novel_story_plus.csv', index_col=0)

In [37]:
test_1 = test['story']
test_2 = test['review']
test_3 = test['piece']
test_4 = test['story'] + test['review'] + test['piece']

In [9]:
tokenizer_bert_kor_base = BertTokenizerFast.from_pretrained("kykim/albert-kor-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizerFast'.


In [41]:
titles_t_1 = test_1.to_numpy().reshape(-1,1)
titles_t_2 = test_2.to_numpy().reshape(-1,1)
titles_t_3 = test_3.to_numpy().reshape(-1,1)
titles_t_4 = test_4.to_numpy().reshape(-1,1)

# oversample = RandomOverSampler()
# X_over, y_over = oversample.fit_resample(titles_t, labels_t)
# train = pd.DataFrame({'title':X_over.reshape(-1), 'topic_idx':y_over.reshape(-1)})
test_1 = pd.DataFrame({'title':titles_t_1.reshape(-1)})
test_2 = pd.DataFrame({'title':titles_t_2.reshape(-1)})
test_3 = pd.DataFrame({'title':titles_t_3.reshape(-1)})
test_4 = pd.DataFrame({'title':titles_t_4.reshape(-1)})

In [11]:

# 뉴스 데이터 클래스 생성
class NewsSubjectDataset(Dataset):
  def __init__(self, subjects, targets, tokenizer, max_len):
    self.subjects = subjects
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  def __len__(self):
    return len(self.subjects)
  def __getitem__(self, item):
    subject = str(self.subjects[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      subject,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding = 'max_length',
      truncation = True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'subject_text': subject,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }
def create_data_loader(df, tokenizer, max_len, batch_size, shuffle_=False, valid=False):
  if valid:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=np.zeros(len(df)),
      tokenizer=tokenizer,
      max_len=max_len
      )
  else:
    ds = NewsSubjectDataset(
      subjects=df.title.to_numpy(),
      targets=df.topic_idx.to_numpy(),
      tokenizer=tokenizer,
      max_len=max_len
    )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4,
    shuffle = shuffle_
  )
  


In [42]:
# 데이터 로더 생성
BATCH_SIZE =64
MAX_LEN =32
# train_data_loader = create_data_loader(train, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, shuffle_=True)
valid_data_loader_1 = create_data_loader(test_1, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)
valid_data_loader_2 = create_data_loader(test_2, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)
valid_data_loader_3 = create_data_loader(test_3, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)
valid_data_loader = create_data_loader(test_4, tokenizer_bert_kor_base, MAX_LEN, BATCH_SIZE, valid=True)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
import random
class NewsSubjectClassifier(nn.Module):
  def __init__(self, n_classes):
    super(NewsSubjectClassifier, self).__init__()
    self.bert = AlbertModel.from_pretrained("kykim/albert-kor-base")
    self.drop = nn.Dropout(p=0.5)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
       return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

device = torch.device("cpu")

def get_predictions(model, data_loader):
  model = model.eval()
  subject_texts = []
  predictions = []
  prediction_probs = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["subject_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      subject_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  return subject_texts, predictions, prediction_probs


In [15]:
model = torch.load(PATH+"model_albert_kor_base.pth")


In [22]:
subject_texts_1, predictions_1, prediction_probs_1 = get_predictions(model, valid_data_loader_1)
subject_texts_2, predictions_2, prediction_probs_2 = get_predictions(model, valid_data_loader_2)
subject_texts_3, predictions_3, prediction_probs_3 = get_predictions(model, valid_data_loader_3)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [44]:

ls1 = []
for i in prediction_probs_1:
  ls1.append(i[:6])

ls2 = []
for i in prediction_probs_2:
  ls2.append(i[:6])

ls3 = []
for i in prediction_probs_3:
  ls3.append(i[:6])

In [43]:
subject_texts, predictions, prediction_probs = get_predictions(model, valid_data_loader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [45]:

ls = []
for i in prediction_probs:
  ls.append(i[:6])


In [ ]:
prediction_probs_1

In [46]:
df_story = pd.DataFrame({'story':subject_texts_1, 'story_predict':predictions_1, 'predict_probs':ls1}) 
df_review = pd.DataFrame({'review':subject_texts_2, 'review_predict':predictions_2, 'predict_probs':ls2})
df_piece = pd.DataFrame({'piece':subject_texts_3, 'piece_predict':predictions_3, 'predict_probs':ls3})
df_all = pd.DataFrame({'piece':subject_texts, 'piece_predict':predictions, 'predict_probs':ls})

In [58]:
# df_story.to_csv('/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/novel_story_감성분류.csv')
# df_review.to_csv('/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/novel_review_감성분류.csv')
# df_piece.to_csv('/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/novel_piece_감성분류.csv')
df_all.to_csv('/content/drive/My Drive/3조/DS_folder/jupyter notebook/Data/novel_전체_감성분류.csv')

In [62]:
df_all

,piece,piece_predict,predict_probs
0,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,5,"[tensor(3.6035), tensor(1.1035), tensor(3.5023..."
1,어디에도 속하지 못했던 자이니치들의 분노와 슬픔에서 탄생한 대작!한국계 1.5세인 ...,5,"[tensor(3.6035), tensor(1.1035), tensor(3.5023..."
2,"※ 인터넷 한정 특별판: 매장 구매, 바로드림 구매 시에는 다른 표지의 에디션으로 ...",2,"[tensor(4.2230), tensor(1.4940), tensor(4.8842..."
3,nan,2,"[tensor(2.9042), tensor(2.4138), tensor(3.1354..."
4,괴물인 내가 또 다른 괴물을 만났다!영화와도 같은 강렬한 사건과 매혹적인 문체로 시...,3,"[tensor(1.9092), tensor(3.0089), tensor(2.4752..."
...,...,...,...
1113,nan,2,"[tensor(2.9042), tensor(2.4138), tensor(3.1354..."
1114,"출장 중 우연히 알게 된 신기한 사실, 그리고 그 비밀로 인해 한 평범한 직장인의 ...",2,"[tensor(1.7681), tensor(2.1924), tensor(4.5782..."
1115,“나는 이 사랑 때문에 죽을 거야.”바닥까지 가라앉았던 마음이 순식간에 목 끝까지 ...,0,"[tensor(6.0404), tensor(2.0551), tensor(3.0429..."
1116,정낙추 소설집『복자는 울지 않았다』. 개발과 이윤의 논리에 훼손당한 이 시대의 자화...,4,"[tensor(1.2628), tensor(1.3547), tensor(4.4286..."
